In [1]:
import json
import os
import timeit

import numpy as np
import pandas as pd
import string

from scipy import stats

import nltk

### load samples

In [6]:
data_dir = ".."
train_path = os.path.join(data_dir, "training.json")
dev_path = os.path.join(data_dir, "development.json")
test_path = os.path.join(data_dir, "test.json")

In [9]:
with open(train_path) as fin:
    j = json.load(fin)
    
train_raw = j["data"]

with open(dev_path) as fin:
    j = json.load(fin)
    
dev_raw = j["data"]

### count of stop words

In [13]:
tokset_stopwords = set(nltk.corpus.stopwords.words("english"))
tokset_punct = set(string.punctuation)

tokset_stopwords.update(tokset_punct)

In [15]:
from collections import Counter

c0 = Counter()
tot0 = 0
c1 = Counter()
tot1 = 0

i = 0

for passage in data:
    title = passage['title']
    paragraphs = passage['paragraphs']
    for paragraph in paragraphs:
        context = paragraph['context']
        qas = paragraph['qas']
        for qa in qas:
            question = qa['question']
            tokset_question = set(nltk.tokenize.word_tokenize(question.lower()))
            qid = qa['id']
            
            c = Counter((tok for tok in tokset_question if tok in tokset_stopwords))
            
            label = int(qa["is_impossible"])
            if label == 1:
                c1 += c
                tot1 += 1
            else:
                c0 += c
                tot0 += 1

            if i % 1000 == 0:
                print(i)
            i += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000


In [22]:
keys = set(c0.keys())
keys.update(c1.keys())
keys = sorted(keys)

for k in keys:
    print(f"{k:>10s} : {c0[k]/tot0:.4f}, {c1[k]/tot1:.4f}, {c0[k]/tot0-c1[k]/tot1:.4f}")

         ! : 0.0002, 0.0001, 0.0001
         # : 0.0001, 0.0003, -0.0001
         $ : 0.0005, 0.0013, -0.0008
         % : 0.0016, 0.0037, -0.0020
         & : 0.0014, 0.0012, 0.0002
         ' : 0.0094, 0.0043, 0.0051
         ( : 0.0014, 0.0011, 0.0003
         ) : 0.0014, 0.0011, 0.0002
         * : 0.0000, 0.0000, 0.0000
         + : 0.0001, 0.0001, -0.0000
         , : 0.0672, 0.0397, 0.0274
         - : 0.0003, 0.0003, 0.0001
         . : 0.0042, 0.0023, 0.0019
         / : 0.0000, 0.0000, 0.0000
         : : 0.0007, 0.0008, -0.0001
         ; : 0.0003, 0.0003, -0.0000
         < : 0.0000, 0.0000, 0.0000
         = : 0.0000, 0.0001, -0.0000
         > : 0.0002, 0.0001, 0.0001
         ? : 0.9892, 0.9922, -0.0030
         [ : 0.0001, 0.0000, 0.0001
         \ : 0.0000, 0.0000, -0.0000
         ] : 0.0001, 0.0000, 0.0001
         ` : 0.0000, 0.0000, 0.0000
         a : 0.1168, 0.1128, 0.0040
     about : 0.0126, 0.0122, 0.0004
     above : 0.0008, 0.0011, -0.0003
     after : 0.016

In [21]:
len(tokset_stopwords)

211

### load word2vec

In [2]:
import gensim

C:\Users\zhenl\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
WORD2VEC_PATH = "we/GoogleNews-vectors-negative300.bin"

In [4]:
w2v = gensim.models.KeyedVectors.load_word2vec_format(WORD2VEC_PATH, binary=True)

In [5]:
we_len = 300
def get_vector(word):
    global w2v, we_len
    return w2v.get_vector(word) if word in w2v.vocab \
            else np.zeros((we_len, ))